# Shot Boundery Detection by VGG16 vetcor Cloustering

In [7]:
import os

import multiprocessing
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from progressbar import progressbar

def show(img, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.show()

In [8]:
import itertools
import numpy as np
import pandas as pd
import networkx as nx

from natsort import natsorted, ns
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

import community

In [9]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [10]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Predict for all images

In [13]:
"""if u are using GPU"""
gpu = True
if(gpu): num_cores = 1
else:    num_cores = multiprocessing.cpu_count()

In [14]:
# define model
model_vgg16 = model

# define input
filenames = os.listdir('./data/')[::1]

# define prediction function
def vgg16Pred(fname):
    try:
        img = image.load_img('data/'+fname, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)

        vgg16_feature = model_vgg16.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)

        return vgg16_feature_np.flatten()
    except: pass
    
vgg16_feature_list = Parallel(n_jobs=num_cores)(delayed(vgg16Pred)(fname) for fname in progressbar(filenames))
vgg16_feature_list_np = np.array(vgg16_feature_list)
vgg16_feature_list_np.shape

100% (275 of 275) |######################| Elapsed Time: 0:03:27 Time:  0:03:27


(275, 25088)

## Vec 2 Graph

In [15]:
A =  vgg16_feature_list_np
A_sparse = sparse.csr_matrix(A)

similarities = cosine_similarity(A_sparse)

In [16]:
G = nx.Graph()

fsize = len(filenames)
for i in progressbar(range(fsize-1)):
    for j in range(i, fsize):
        u,v = filenames[i], filenames[j]
        G.add_edge(u, v, weight = 1 - similarities[i][j])
               


100% (274 of 274) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [23]:
# Get best partition
partition = community.best_partition(G)
print('Modularity: ', community.modularity(partition, G))

Modularity:  0.0


In [35]:
g

NameError: name 'g' is not defined